# Optimising Portfolio Weights of 10 Stocks

In [1]:
# Import package dependencies
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [3]:
df1 = pd.read_csv('AAPL.csv')
df2 = pd.read_csv('ARKG.csv')
df3 = pd.read_csv('ARKK.csv')
df4 = pd.read_csv('EEM.csv')
df5 = pd.read_csv('GOOGL.csv')
df6 = pd.read_csv('ICLN.csv')
df7 = pd.read_csv('MSFT.csv')
df8 = pd.read_csv('QQQ.csv')
df9 = pd.read_csv('VOO.csv')
df10 = pd.read_csv('VWO.csv')

In [4]:
df1 = df1[['Date', 'Adj Close']]
df1.rename(columns={'Date' : 'date', 'Adj Close' : 'AAPL'}, inplace=True)
df1.set_index('date', inplace=True)

df2 = df2[['Date', 'Adj Close']]
df2.rename(columns={'Date' : 'date', 'Adj Close' : 'ARKG'}, inplace=True)
df2.set_index('date', inplace=True)

df3 = df3[['Date', 'Adj Close']]
df3.rename(columns={'Date' : 'date', 'Adj Close' : 'ARKK'}, inplace=True)
df3.set_index('date', inplace=True)

df4 = df4[['Date', 'Adj Close']]
df4.rename(columns={'Date' : 'date', 'Adj Close' : 'EEM'}, inplace=True)
df4.set_index('date', inplace=True)

df5 = df5[['Date', 'Adj Close']]
df5.rename(columns={'Date' : 'date', 'Adj Close' : 'GOOGL'}, inplace=True)
df5.set_index('date', inplace=True)

df6 = df6[['Date', 'Adj Close']]
df6.rename(columns={'Date' : 'date', 'Adj Close' : 'ICLN'}, inplace=True)
df6.set_index('date', inplace=True)

df7 = df7[['Date', 'Adj Close']]
df7.rename(columns={'Date' : 'date', 'Adj Close' : 'MSFT'}, inplace=True)
df7.set_index('date', inplace=True)

df8 = df8[['Date', 'Adj Close']]
df8.rename(columns={'Date' : 'date', 'Adj Close' : 'QQQ'}, inplace=True)
df8.set_index('date', inplace=True)

df9 = df9[['Date', 'Adj Close']]
df9.rename(columns={'Date' : 'date', 'Adj Close' : 'VOO'}, inplace=True)
df9.set_index('date', inplace=True)

df10 = df10[['Date', 'Adj Close']]
df10.rename(columns={'Date' : 'date', 'Adj Close' : 'VWO'}, inplace=True)
df10.set_index('date', inplace=True)

df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10],axis=1)
df.head()

,AAPL,ARKG,ARKK,EEM,GOOGL,ICLN,MSFT,QQQ,VOO,VWO
date,,,,,,,,,,
2016-06-06,23.034662,17.126883,18.684488,30.824535,730.059998,8.119722,48.025192,106.049301,176.107620,30.556597
2016-06-07,23.128084,17.015066,18.693867,31.103611,731.090027,8.164486,47.997551,105.780472,176.425583,30.854626
2016-06-08,23.107069,16.940521,18.684488,31.328676,742.929993,8.164486,47.942272,105.962898,177.007004,31.108820
2016-06-09,23.272882,16.527723,18.412472,30.968580,742.520020,8.074963,47.555336,105.790085,176.734497,30.758200
2016-06-10,23.081377,16.157789,18.384335,30.185362,733.190002,7.734777,47.426357,104.589989,175.035614,29.978071


In [5]:
returns_df = df.pct_change(1).dropna()
returns_df.head()

,AAPL,ARKG,ARKK,EEM,GOOGL,ICLN,MSFT,QQQ,VOO,VWO
date,,,,,,,,,,
2016-06-07,0.004056,-0.006529,0.000502,0.009054,0.001411,0.005513,-0.000576,-0.002535,0.001806,0.009753
2016-06-08,-0.000909,-0.004381,-0.000502,0.007236,0.016195,0.000000,-0.001152,0.001725,0.003296,0.008238
2016-06-09,0.007176,-0.024367,-0.014558,-0.011494,-0.000552,-0.010965,-0.008071,-0.001631,-0.001540,-0.011271
2016-06-10,-0.008229,-0.022383,-0.001528,-0.025291,-0.012565,-0.042128,-0.002712,-0.011344,-0.009613,-0.025363
2016-06-13,-0.015077,-0.012111,-0.014796,-0.014017,-0.001787,-0.028935,-0.026029,-0.008353,-0.007526,-0.012573


In [7]:
returns_df.mean()

AAPL     0.001533
ARKG     0.001488
ARKK     0.001627
EEM      0.000571
GOOGL    0.001087
ICLN     0.000951
MSFT     0.001464
QQQ      0.001015
VOO      0.000702
VWO      0.000555
dtype: float64

In [9]:
returns_df9 = df9.pct_change(1).dropna() #using df9 because it is VOO, basically tracking the whole US market
annual_market_return = returns_df9.mean() * 250
annual_market_return

VOO    0.17555
dtype: float64

In [13]:
#annual market risk calculation
annual_market_risk = np.std(returns_df9) * np.sqrt(250)
annual_market_risk

VOO    0.190492
dtype: float64

In [14]:
def getPortReturn(weights): #calculating the expected return on the portfolio
    """
    Returns the Annualised Expected Return of a portfolio.
    Annualises the return using the 'crude' method.
    """
    exp_ret_portfolio = np.dot(np.transpose(weights), returns_df.mean()) * 250
    
    return exp_ret_portfolio

In [15]:
# Create a vector of equal weights as the initial (init) guess
num_stocks = len(returns_df.columns)
init_weights = [1 / num_stocks] * num_stocks
init_weights

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [16]:
# Calculate the expected return on the portfolio of equal weights
getPortReturn(init_weights) #annualised return of 27.5%

0.27480030648761644

In [17]:
bounds = tuple((0,1) for i in range(num_stocks))
cons = ({'type' :'eq', 'fun' : lambda w : np.sum(w) -1},
        {'type' :'eq', 'fun' : lambda x : x.dot(returns_df.mean())*250 - annual_market_return}) 

In [18]:
results = minimize (fun=getPortReturn, x0=init_weights, bounds=bounds, constraints=cons)
results #pretty close to the return of VOO

     fun: 0.17555020244940803
     jac: array([0.38317505, 0.37188333, 0.40664976, 0.14283642, 0.27167399,
       0.23779757, 0.36606871, 0.25365851, 0.1755502 , 0.13870952])
 message: 'Optimization terminated successfully.'
    nfev: 24
     nit: 2
    njev: 2
  status: 0
 success: True
       x: array([4.16333634e-17, 1.80411242e-16, 0.00000000e+00, 2.65739401e-01,
       5.48800443e-02, 1.10323185e-01, 0.00000000e+00, 8.43647217e-02,
       2.12199055e-01, 2.72493593e-01])

In [58]:
optimised_weights = pd.DataFrame(results['x'])
optimised_weights.index = returns_df.columns
optimised_weights.rename(columns={optimised_weights.columns[0] : 'weights_market_return'}, inplace=True)
optimised_weights['weights_market_return'] = optimised_weights['weights_market_return'].apply(lambda x : round(x, 3))
optimised_weights

,weights_market_return
AAPL,0.000
ARKG,0.000
ARKK,0.000
EEM,0.266
GOOGL,0.055
ICLN,0.110
MSFT,0.000
QQQ,0.084
VOO,0.212
VWO,0.272


In [34]:
vcv_matrix = returns_df.cov()
var_p = np.dot(np.transpose(optimised_weights), np.dot(vcv_matrix,optimised_weights))
portfolio_risk_annual = np.sqrt(var_p) * np.sqrt(250)
portfolio_risk_annual #risk is 19.97% when we are trying to match the market portfolio

array([[0.19972871]])

In [48]:
def getPortRisk(weights): #weights is the parameter, we are creating objective function
    
    '''Returns the annualised standard deviation of a k asset portfolio.'''

    returns_df = df.pct_change(1).dropna()  # estimate returns for each asset
    num_stocks = len(returns_df.columns)  # number of stocks based on number of columns (excluding index col)
                                          # this is a local variable
        
    vcv = returns_df.cov()  # being the variance covariance matrix
    
    var_p = np.dot(np.transpose(weights), np.dot(vcv, weights))  # variance of the multi-asset portfolio
    sd_p = np.sqrt(var_p)  # standard deviation of the multi-asset portfolio
    sd_p_annual = sd_p * np.sqrt(250)  # annualised standard deviation of the multi-asset portfolio
    
    return sd_p_annual

In [49]:
cons2 = ({'type' :'eq', 'fun' : lambda y : np.sum(y) -1})

In [50]:
results2 = minimize(fun=getPortRisk, x0=init_weights, bounds=bounds, constraints=cons2)

In [51]:
results2

     fun: 0.18708227226097046
     jac: array([0.22367729, 0.23015593, 0.24736513, 0.19685099, 0.20402578,
       0.20416494, 0.2245053 , 0.20235765, 0.1870823 , 0.18708221])
 message: 'Optimization terminated successfully.'
    nfev: 144
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([1.59770351e-17, 3.23850534e-17, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 1.96143040e-17, 0.00000000e+00, 0.00000000e+00,
       7.14190116e-01, 2.85809884e-01])

In [52]:
getPortRisk(init_weights)

0.22448477321542992

In [61]:
optimised_weights2 = pd.DataFrame(results2['x'])
optimised_weights2.index = df.columns
optimised_weights2.rename(columns={optimised_weights2.columns[0] : 'weights_minimized_risk'}, inplace=True)
optimised_weights2['weights_minimized_risk'] = optimised_weights2['weights_minimized_risk'].apply(lambda x : round(x, 3))
optimised_weights2

,weights_minimized_risk
AAPL,0.000
ARKG,0.000
ARKK,0.000
EEM,0.000
GOOGL,0.000
ICLN,0.000
MSFT,0.000
QQQ,0.000
VOO,0.714
VWO,0.286


In [62]:
Table = pd.concat([optimised_weights,optimised_weights2],axis=1)
Table

,weights_market_return,weights_minimized_risk
AAPL,0.000,0.000
ARKG,0.000,0.000
ARKK,0.000,0.000
EEM,0.266,0.000
GOOGL,0.055,0.000
ICLN,0.110,0.000
MSFT,0.000,0.000
QQQ,0.084,0.000
VOO,0.212,0.714
VWO,0.272,0.286
